## **Processing Text**

In [39]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim
import gensim.downloader as api
import json 
import ast

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df= pd.read_csv('/content/drive/MyDrive/Myntra Fasion Clothing.csv')
df.head()

<ipython-input-3-9892349772c1>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('/content/drive/MyDrive/Myntra Fasion Clothing.csv')


,URL,Product_id,BrandName,Category,Individual_category,category_by_Gender,Description,DiscountPrice (in Rs),OriginalPrice (in Rs),DiscountOffer,SizeOption,Ratings,Reviews
0,https://www.myntra.com/jeans/roadster/roadster...,2296012,Roadster,Bottom Wear,jeans,Men,roadster men navy blue slim fit mid rise clean...,824.0,1499.0,45% OFF,"28, 30, 32, 34, 36",3.9,999.0
1,https://www.myntra.com/track-pants/locomotive/...,13780156,LOCOMOTIVE,Bottom Wear,track-pants,Men,locomotive men black white solid slim fit tra...,517.0,1149.0,55% OFF,"S, M, L, XL",4.0,999.0
2,https://www.myntra.com/shirts/roadster/roadste...,11895958,Roadster,Topwear,shirts,Men,roadster men navy white black geometric print...,629.0,1399.0,55% OFF,"38, 40, 42, 44, 46, 48",4.3,999.0
3,https://www.myntra.com/shapewear/zivame/zivame...,4335679,Zivame,Lingerie & Sleep Wear,shapewear,Women,zivame women black saree shapewear zi3023core0...,893.0,1295.0,31% OFF,"S, M, L, XL, XXL",4.2,999.0
4,https://www.myntra.com/tshirts/roadster/roadst...,11690882,Roadster,Western,tshirts,Women,roadster women white solid v neck pure cotton ...,NaN,599.0,35% OFF,"XS, S, M, L, XL",4.2,999.0


In [4]:
def preprocess_txt(text):
  # convert all words to lower case
  text = text.lower()
  # remove all punctuations 
  text_p = "".join([char for char in text if char not in string.punctuation])
  # tokenize the sentence
  words = word_tokenize(text_p)
  # remove stop words of english corpora
  stop_words = stopwords.words('english')
  filtered_words = [word for word in words if word not in stop_words]
  # remove duplicate words
  final_text = list(set(filtered_words))
  return final_text

In [5]:
# process the product description column and store it in a new column
df['processed_text'] = [' '.join(word for word in preprocess_txt(s)) for s in list(df['Description'])]
df['processed_text']

0         rise fit men clean jeans blue mid roadster nav...
1         fit men white locomotive pants black slim trac...
2         men white printed shirt roadster navy sustaina...
3         zivame shapewear saree women black zi3023core0...
4         white pure cotton shirt women roadster neck v ...
                                ...                        
526558             hm jumper beige cashmere women oversized
526559          jacket women water black parka hm repellent
526560                   250 sheath lace blue dress designs
526561               printed dress line women yellow masaba
526562             blue shirt women sleeves jockey extended
Name: processed_text, Length: 526563, dtype: object

In [6]:
sentences = list(df['processed_text'].values)
sentences

['rise fit men clean jeans blue mid roadster navy slim look',
 'fit men white locomotive pants black slim track solid',
 'men white printed shirt roadster navy sustainable black casual geometric',
 'zivame shapewear saree women black zi3023core0nude',
 'white pure cotton shirt women roadster neck v solid',
 'mast harbour women yellow tank top solid',
 'fit men highlander trousers olive regular slim green solid',
 'pure cotton line top pink mayra embroidered',
 '2 pack shirts women roadster solid',
 'rise fit men clean herenow jeans blue mid slim stretchable ankle look length',
 'men dry hrithik roshan hrx tights rapid training',
 'polo men white pure striped cotton blue shirt roadster navy collar',
 'anubhutee printed white trousers dupatta women kurta pink',
 'athena women black basic solid jumpsuit',
 'shirt round women roadster maroon neck solid',
 'fit men highlander printed blue shirt black slim casual',
 'floral blue line women kurta pink vishudh print',
 'white printed sangria d

## **Create GloVe embeddings of product description**

In [42]:
# load pretrained glove vectors on wikipedia
glove_wiki = api.load("glove-wiki-gigaword-50")


def get_GloVe(text, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size)) # create for size of glove embedding and assign all values 0
    count = 0
    for word in text:
        try:
            vec += vectors[word].reshape((1, size)) # update vector with new word
            count += 1 # counts every word in sentence
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count  #get average of vector to create embedding for sentence
        return vec
    elif aggregation == 'sum':
        return vec

In [18]:
# create emdeddings column
df['GloVE'] = [get_GloVe(s, 50, glove_wiki,'mean').tolist()[0] for s in sentences]

In [19]:
df.head()

,URL,Product_id,BrandName,Category,Individual_category,category_by_Gender,Description,DiscountPrice (in Rs),OriginalPrice (in Rs),DiscountOffer,SizeOption,Ratings,Reviews,processed_text,GloVE
0,https://www.myntra.com/jeans/roadster/roadster...,2296012,Roadster,Bottom Wear,jeans,Men,roadster men navy blue slim fit mid rise clean...,824.0,1499.0,45% OFF,"28, 30, 32, 34, 36",3.9,999.0,rise fit men clean jeans blue mid roadster nav...,"[-0.23400302375243467, 0.051322002302516594, 0..."
1,https://www.myntra.com/track-pants/locomotive/...,13780156,LOCOMOTIVE,Bottom Wear,track-pants,Men,locomotive men black white solid slim fit tra...,517.0,1149.0,55% OFF,"S, M, L, XL",4.0,999.0,fit men white locomotive pants black slim trac...,"[-0.4489755539430512, 0.25248443914784324, -0...."
2,https://www.myntra.com/shirts/roadster/roadste...,11895958,Roadster,Topwear,shirts,Men,roadster men navy white black geometric print...,629.0,1399.0,55% OFF,"38, 40, 42, 44, 46, 48",4.3,999.0,men white printed shirt roadster navy sustaina...,"[-0.3818101976066828, 0.30517099481076004, -0...."
3,https://www.myntra.com/shapewear/zivame/zivame...,4335679,Zivame,Lingerie & Sleep Wear,shapewear,Women,zivame women black saree shapewear zi3023core0...,893.0,1295.0,31% OFF,"S, M, L, XL, XXL",4.2,999.0,zivame shapewear saree women black zi3023core0...,"[-0.929332509636879, 0.1168774962425232, -0.74..."
4,https://www.myntra.com/tshirts/roadster/roadst...,11690882,Roadster,Western,tshirts,Women,roadster women white solid v neck pure cotton ...,NaN,599.0,35% OFF,"XS, S, M, L, XL",4.2,999.0,white pure cotton shirt women roadster neck v ...,"[-0.6888510998752382, 0.42917444474167293, -0...."


In [20]:
df.to_csv('Glove Embeddings.csv')

In [52]:
df=pd.read_csv('/content/Glove Embeddings.csv')

<ipython-input-52-335274ebf989>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/Glove Embeddings.csv')


# **Streamlit script**

In [28]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=7f4c13b9d3be14aff74d0c32cfe694c37c877a654df480f409f6dd546faf9c8d
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [ ]:
%%writefile similarity.py
import streamlit as st


def main():

    import pandas as pd
    import numpy as np
    import nltk
    import string
    from nltk import word_tokenize
    from nltk.corpus import stopwords
    import gensim
    import gensim.downloader as api
    import ast
    import json

    def preprocess_txt(text):
      # convert all words to lower case
      text = text.lower()
      # remove all punctuations 
      text_p = "".join([char for char in text if char not in string.punctuation])
      # tokenize the sentence
      words = word_tokenize(text_p)
      # remove stop words of english corpora
      stop_words = stopwords.words('english')
      filtered_words = [word for word in words if word not in stop_words]
      # remove duplicate words
      final_text = list(set(filtered_words))
      return final_text


    def cos_similarity(v1,v2):
      return abs(np.dot(v1,v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))

    glove_wiki = api.load("glove-wiki-gigaword-50")

    def get_GloVe(text, size, vectors, aggregation='mean'):
      vec = np.zeros(size).reshape((1, size)) # create for size of glove embedding and assign all values 0
      count = 0
      for word in text:
          try:
              vec += vectors[word].reshape((1, size)) # update vector with new word
              count += 1 # counts every word in sentence
          except KeyError:
              continue
      if aggregation == 'mean':
          if count != 0:
              vec /= count  #get average of vector to create embedding for sentence
          return vec
      elif aggregation == 'sum':
          return vec

    def similar_results(df, target, glove_wiki):
      # target needs to be preprocessed in the same way as the product descriptions were processed
      if isinstance(target,str):
        target = preprocess_txt(target)
      #convert the processed target into embeddings
      vector = get_GloVe(target, 50, glove_wiki,'mean')
      df_ = df[['URL','processed_text','GloVE']]
      # store the cosine similarity scores for each product with the target in a new column
      df_['similarity'] = [cos_similarity(vector[0], ast.literal_eval(df_['GloVE'][i])) for i in range(df_.shape[0])]
      # sort the similarity scores in descending order
      df_.sort_values(by='similarity', ascending = False, inplace=True)
      # convert result into json
      url_dict = df_[['URL']].iloc[:10].to_dict()
      json_object = json.dumps(url_dict, indent=4)
      return json_object


    df = pd.read_csv('/content/Glove Embeddings.csv')
    
    st.title("Clothing Recomendations App")
    html_temp = """
    <div style="background-color:tomato;padding:10px">
    <h2 style="color:white;text-align:center;">You may also like these!</h2>
    </div>
    """
    st.markdown(html_temp,unsafe_allow_html=True)
    target = st.text_input("Input Text","Type Here")
    result=""
    if st.button("Search"):
        result=similar_results(df, target, glove_wiki)
    st.success('The output is {}'.format(result))

if __name__=='__main__':
    main()

In [24]:
!npm install localtunnel


npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.611s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [59]:
!streamlit run /content/similarity.py &>/content/logs.txt &

In [60]:
# fetch the ip address
!curl ipecho.net/plain

35.236.136.55

In [61]:
# get your url
!npx localtunnel --port 8501

npx: installed 22 in 1.911s
your url is: https://dirty-sloths-float.loca.lt
^C
